In [2]:
pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.1 MB/s eta 0:00:00


In [4]:
# match_queries.py
# ------------------------------------------------------------
# Task: Match new user queries to resolved queries using:
#  A) Fuzzy search (RapidFuzz)
#  B) TF-IDF + cosine similarity
#
# Input files:
#   /mnt/data/resolved_queries.csv  -> columns: [Query_ID, Pre_Resolved_Query]
#   /mnt/data/new_queries.csv       -> columns: [Variation_Query, Matches_With_Query_ID] (ground truth)
#
# Output:
#   matches_results.csv  -> combined predictions + scores
#   Also prints accuracy metrics and suggested thresholds.
# ------------------------------------------------------------

import re
import string
import numpy as np
import pandas as pd

# --- Try to use rapidfuzz (faster, better than fuzzywuzzy). Fallback to fuzzywuzzy if unavailable.
try:
    from rapidfuzz import fuzz, process
    _USE_RAPIDFUZZ = True
except Exception:
    _USE_RAPIDFUZZ = False
    from fuzzywuzzy import fuzz, process  # type: ignore

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# =========================
# Config
# =========================
PATH_RESOLVED = "/content/resolved_queries.csv"
PATH_NEW = "/content/new_queries.csv"
OUTPUT_PATH = "matches_results.csv"

# If you want to control TF-IDF ngram range or stopwords, tweak here:
TFIDF_NGRAM_RANGE = (1, 2)
TFIDF_MIN_DF = 1
TFIDF_MAX_FEATURES = None   # or e.g., 5000


# =========================
# Preprocessing
# =========================
_punc_table = str.maketrans("", "", string.punctuation)

def clean_text(s: str) -> str:
    """Light normalization that helps both fuzzy and TF-IDF."""
    if not isinstance(s, str):
        s = "" if s is None else str(s)
    s = s.lower()
    s = s.translate(_punc_table)              # remove punctuation
    s = re.sub(r"\s+", " ", s).strip()        # collapse whitespace
    # simple normalizations
    s = s.replace("cant", "can not").replace("cant ", "can not ")
    s = s.replace("won't", "will not").replace("wont", "will not")
    s = s.replace("can't", "can not")
    s = s.replace("dont", "do not").replace("don't", "do not")
    s = s.replace("checkout", "check out")
    return s


# =========================
# Load Data
# =========================
resolved_df = pd.read_csv(PATH_RESOLVED)
new_df = pd.read_csv(PATH_NEW)

# Clean copies used for matching
resolved_df["clean"] = resolved_df["Pre_Resolved_Query"].apply(clean_text)
new_df["clean"] = new_df["Variation_Query"].apply(clean_text)

# Handy lookup
id2query = dict(zip(resolved_df["Query_ID"], resolved_df["Pre_Resolved_Query"]))
id2clean = dict(zip(resolved_df["Query_ID"], resolved_df["clean"]))
query_ids = resolved_df["Query_ID"].tolist()
resolved_clean_list = resolved_df["clean"].tolist()


# =========================
# A) Fuzzy Matching
# =========================
def fuzzy_best_match(text: str, candidates: list[str]):
    """
    Returns (best_text, score) using a token-based scorer.
    RapidFuzz score is 0..100. FuzzyWuzzy same scale.
    """
    # A robust scorer for noisy variations:
    # token_set_ratio handles reordered tokens / extras well.
    scorer = fuzz.token_set_ratio
    if _USE_RAPIDFUZZ:
        best = process.extractOne(text, candidates, scorer=scorer)
        # best -> (match_text, score, idx)
        return best[0], float(best[1]), int(best[2])
    else:
        # fuzzywuzzy's process.extractOne returns (match_text, score)
        match_text, score = process.extractOne(text, candidates, scorer=scorer)
        idx = candidates.index(match_text)
        return match_text, float(score), int(idx)


# Compute best fuzzy match per variation
fuzzy_matches = []
for q in new_df["clean"].tolist():
    best_text, score, idx = fuzzy_best_match(q, resolved_clean_list)
    best_id = query_ids[idx]
    fuzzy_matches.append((best_id, score))

new_df["Fuzzy_Pred_ID"] = [m[0] for m in fuzzy_matches]
new_df["Fuzzy_Score"] = [m[1] for m in fuzzy_matches]


# =========================
# B) TF-IDF + Cosine
# =========================
# Fit only on resolved queries; transform variations on the same vectorizer.
vectorizer = TfidfVectorizer(ngram_range=TFIDF_NGRAM_RANGE,
                             min_df=TFIDF_MIN_DF,
                             max_features=TFIDF_MAX_FEATURES)
X_resolved = vectorizer.fit_transform(resolved_df["clean"])
X_new = vectorizer.transform(new_df["clean"])

# For each new row, pick the resolved with highest cosine
cosine_scores = cosine_similarity(X_new, X_resolved)  # shape: [n_new, n_resolved]
tfidf_pred_idx = np.argmax(cosine_scores, axis=1)
tfidf_best_scores = cosine_scores[np.arange(cosine_scores.shape[0]), tfidf_pred_idx]
tfidf_pred_ids = [query_ids[i] for i in tfidf_pred_idx]

new_df["TFIDF_Pred_ID"] = tfidf_pred_ids
new_df["Cosine_Score"] = tfidf_best_scores


# =========================
# Evaluation & Threshold Suggestion
# =========================
def top1_accuracy(pred_col: str, truth_col: str, df: pd.DataFrame) -> float:
    return float((df[pred_col] == df[truth_col]).mean())

def thresholded_accuracy(score_col: str, pred_col: str, truth_col: str, df: pd.DataFrame, threshold: float, score_scale: str) -> tuple[float, float]:
    """
    Returns (acc_on_matched, coverage).
      - Only count rows where score >= threshold as 'attempted' matches.
      - acc_on_matched = accuracy restricted to attempted rows (NaN -> 0 if none).
      - coverage = attempted / total.
    score_scale: "0-100" for fuzzy, "0-1" for cosine
    """
    if score_scale == "0-100":
        mask = df[score_col] >= threshold
    else:  # 0-1
        mask = df[score_col] >= threshold

    if mask.sum() == 0:
        return 0.0, 0.0
    acc = float((df.loc[mask, pred_col] == df.loc[mask, truth_col]).mean())
    coverage = float(mask.mean())
    return acc, coverage


# Compute raw top-1 accuracies (no thresholds)
acc_fuzzy = top1_accuracy("Fuzzy_Pred_ID", "Matches_With_Query_ID", new_df)
acc_tfidf = top1_accuracy("TFIDF_Pred_ID", "Matches_With_Query_ID", new_df)

# Sweep thresholds to recommend a good cutoff
def sweep_thresholds(df: pd.DataFrame):
    # Fuzzy: 0..100
    fuzzy_grid = list(range(50, 101, 5))  # 50,55,...100
    fuzzy_results = []
    for t in fuzzy_grid:
        a, cov = thresholded_accuracy("Fuzzy_Score", "Fuzzy_Pred_ID", "Matches_With_Query_ID", df, t, "0-100")
        fuzzy_results.append((t, a, cov))

    # Cosine: 0..1
    cos_grid = [x / 100 for x in range(10, 101, 5)]  # 0.10,0.15,...,1.00
    cos_results = []
    for t in cos_grid:
        a, cov = thresholded_accuracy("Cosine_Score", "TFIDF_Pred_ID", "Matches_With_Query_ID", df, t, "0-1")
        cos_results.append((t, a, cov))

    # Choose threshold that maximizes (accuracy * coverage) — balances precision & recall
    best_fuzzy = max(fuzzy_results, key=lambda x: x[1] * x[2])
    best_cos = max(cos_results, key=lambda x: x[1] * x[2])
    return fuzzy_results, cos_results, best_fuzzy, best_cos

fuzzy_grid_res, cos_grid_res, best_fuzzy, best_cos = sweep_thresholds(new_df)

# Human-readable summary (printed when you run the script)
print("\n=== Raw Top-1 Accuracies (no threshold) ===")
print(f"Fuzzy top-1 accuracy  : {acc_fuzzy:.3f}")
print(f"TF-IDF top-1 accuracy : {acc_tfidf:.3f}")

print("\n=== Suggested Thresholds (balance = accuracy * coverage) ===")
print(f"Fuzzy  -> threshold={best_fuzzy[0]}  acc_on_matched={best_fuzzy[1]:.3f}  coverage={best_fuzzy[2]:.3f}")
print(f"Cosine -> threshold={best_cos[0]:.2f} acc_on_matched={best_cos[1]:.3f}  coverage={best_cos[2]:.3f}")

# (Optional) If you want a fixed default instead of sweep suggestions:
# DEFAULT_FUZZY_THRESHOLD = 80
# DEFAULT_COSINE_THRESHOLD = 0.35


# =========================
# Show Combined Results
# =========================
cols_out = [
    "Variation_Query",
    "Matches_With_Query_ID",     # ground truth
    "Fuzzy_Pred_ID", "Fuzzy_Score",
    "TFIDF_Pred_ID", "Cosine_Score"
]

print("\n=== Full Results ===")
print(new_df[cols_out].to_string(index=False))   # prints entire df in console neatly

# (Optional) Show top disagreements to inspect:
mismatch = new_df[new_df["Fuzzy_Pred_ID"] != new_df["Matches_With_Query_ID"]].copy()
mismatch = mismatch[["Variation_Query", "Matches_With_Query_ID", "Fuzzy_Pred_ID", "Fuzzy_Score",
                     "TFIDF_Pred_ID", "Cosine_Score"]]
print(f"\n=== Mismatches to inspect (first 10 rows) ===\n{mismatch.head(10)}")




=== Raw Top-1 Accuracies (no threshold) ===
Fuzzy top-1 accuracy  : 1.000
TF-IDF top-1 accuracy : 1.000

=== Suggested Thresholds (balance = accuracy * coverage) ===
Fuzzy  -> threshold=50  acc_on_matched=1.000  coverage=0.950
Cosine -> threshold=0.10 acc_on_matched=1.000  coverage=0.950

=== Full Results ===
                          Variation_Query  Matches_With_Query_ID  Fuzzy_Pred_ID  Fuzzy_Score  TFIDF_Pred_ID  Cosine_Score
         Unabel to conect to the internet                      1              1    94.915254              1      0.752765
                Can’t connect to internet                      1              1    86.363636              1      0.699789
                      Intenet not working                      1              1    48.979592              1      0.000000
             Payment failed while chekout                      2              2    74.576271              2      0.577350
Payment did not go through during chckout                      2              

In [6]:
# match_names.py
# ------------------------------------------------------------
# Task 2: Match person-name variations to a canonical base list
# Methods:
#   A) Fuzzy search (RapidFuzz if available; stdlib difflib fallback)
#   B) TF-IDF + cosine similarity
#
# Inputs:
#   /mnt/data/base_names.csv        -> [Base_Name_ID, Base_Name]
#   /mnt/data/name_variations.csv   -> [Variation, Matches_With_Base_Name] (ground truth)
#
# Output: printed tables & metrics (no files written)
# ------------------------------------------------------------

import re
import unicodedata
import string
import numpy as np
import pandas as pd

# ---- try rapidfuzz; fall back to difflib (stdlib, no install) ----
try:
    from rapidfuzz import fuzz, process
    _USE_RAPIDFUZZ = True
except Exception:
    _USE_RAPIDFUZZ = False
    import difflib

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# =========================
# Config
# =========================
PATH_BASE = "/content/base_names.csv"
PATH_VARIATIONS = "/content/name_variations.csv"

# Conservative production-ish thresholds (we'll also sweep)
FUZZY_THRESHOLD = 85.0          # 0..100
COSINE_THRESHOLD = 0.50         # 0..1

# TF-IDF settings tuned for short strings
TFIDF_NGRAM_RANGE = (1, 3)
TFIDF_MIN_DF = 1
TFIDF_MAX_FEATURES = None


# =========================
# Name Preprocessing
# =========================
_NICKNAMES = {
    # expand as needed
    "bob": "robert",
    "bobby": "robert",
    "rob": "robert",
    "robbie": "robert",
    "mike": "michael",
    "mikey": "michael",
    "bill": "william",
    "billy": "william",
    "will": "william",
    "liz": "elizabeth",
    "beth": "elizabeth",
    "lizzy": "elizabeth",
    "betty": "elizabeth",
    "jen": "jennifer",
    "jenny": "jennifer",
    "tom": "thomas",
    "kat": "katherine",
    "kathy": "katherine",
    "kate": "katherine",
    "tony": "anthony",
    "andy": "andrew",
    "drew": "andrew",
    "chris": "christopher",
    "kris": "christopher",
    "joe": "joseph",
    "joey": "joseph",
    "peggy": "margaret",
    "maggie": "margaret",
    "meg": "megan",   # sometimes margaret; keep simple
    "rick": "richard",
    "dick": "richard",
    "rich": "richard",
    "jim": "james",
    "jimmy": "james",
    "sam": "samantha",  # or samuel; ambiguous
}

_punc_table_keep_space = str.maketrans({c: " " for c in string.punctuation})

def _strip_accents(s: str) -> str:
    return "".join(
        ch for ch in unicodedata.normalize("NFKD", s)
        if not unicodedata.combining(ch)
    )

def normalize_name(s: str) -> str:
    """
    Normalize names to improve fuzzy & TF-IDF matching:
      - lowercase, strip accents
      - handle "last, first [middle]" -> "first last"
      - remove periods in initials (e.g., "J. R. R." -> "J R R")
      - collapse punctuation to spaces, drop apostrophes impact
      - expand simple nicknames token-wise
      - collapse multiple spaces
    """
    if not isinstance(s, str):
        s = "" if s is None else str(s)

    # trim and lowercase
    s = s.strip()
    s = _strip_accents(s).lower()

    # detect "last, first [middle]" -> "first middle last"
    if "," in s:
        parts = [p.strip() for p in s.split(",")]
        if len(parts) >= 2 and parts[0] and parts[1]:
            # move surname to end
            s = parts[1] + " " + parts[0]

    # remove dots in initials, keep letters
    s = s.replace(".", " ")

    # collapse punctuation to spaces (incl. apostrophes in O'Connor)
    s = s.translate(_punc_table_keep_space)

    # collapse whitespace
    s = re.sub(r"\s+", " ", s).strip()

    # expand nicknames token-wise
    tokens = []
    for tok in s.split():
        base = _NICKNAMES.get(tok, tok)
        tokens.append(base)
    s = " ".join(tokens)

    return s


# =========================
# Load Data
# =========================
base_df = pd.read_csv(PATH_BASE)
var_df = pd.read_csv(PATH_VARIATIONS)

base_df["clean"] = base_df["Base_Name"].apply(normalize_name)
var_df["clean"]  = var_df["Variation"].apply(normalize_name)

base_names_clean = base_df["clean"].tolist()
base_names_raw   = base_df["Base_Name"].tolist()
base_ids         = base_df["Base_Name_ID"].tolist()


# =========================
# A) Fuzzy Matching
# =========================
def fuzzy_best(text: str, candidates: list[str]):
    """
    Return (best_idx, score) where score is 0..100
    """
    if _USE_RAPIDFUZZ:
        best = process.extractOne(text, candidates, scorer=fuzz.token_set_ratio)
        # (match_text, score, idx)
        return int(best[2]), float(best[1])
    else:
        # difflib returns 0..1; rescale to 0..100
        ratios = [difflib.SequenceMatcher(None, text, c).ratio() * 100 for c in candidates]
        idx = int(np.argmax(ratios))
        return idx, float(ratios[idx])

fuzzy_idx = []
fuzzy_score = []
for q in var_df["clean"].tolist():
    idx, sc = fuzzy_best(q, base_names_clean)
    fuzzy_idx.append(idx)
    fuzzy_score.append(sc)

var_df["Fuzzy_Pred_Name"] = [base_names_raw[i] for i in fuzzy_idx]
var_df["Fuzzy_Pred_ID"]   = [base_ids[i] for i in fuzzy_idx]
var_df["Fuzzy_Score"]     = fuzzy_score


# =========================
# B) TF-IDF + Cosine
# =========================
vectorizer = TfidfVectorizer(ngram_range=TFIDF_NGRAM_RANGE,
                             min_df=TFIDF_MIN_DF,
                             max_features=TFIDF_MAX_FEATURES)
X_base = vectorizer.fit_transform(base_df["clean"])
X_var  = vectorizer.transform(var_df["clean"])

cos = cosine_similarity(X_var, X_base)  # [n_var, n_base]
cos_idx = np.argmax(cos, axis=1)
cos_best = cos[np.arange(cos.shape[0]), cos_idx]

var_df["TFIDF_Pred_Name"] = [base_names_raw[i] for i in cos_idx]
var_df["TFIDF_Pred_ID"]   = [base_ids[i] for i in cos_idx]
var_df["Cosine_Score"]    = cos_best


# =========================
# Evaluation (against provided ground truth)
# =========================
def top1_acc_name(pred_col: str, truth_col: str, df: pd.DataFrame) -> float:
    # compare normalized strings to be robust to punctuation/case
    pred = df[pred_col].fillna("").map(normalize_name)
    tru  = df[truth_col].fillna("").map(normalize_name)
    return float((pred == tru).mean())

acc_fuzzy  = top1_acc_name("Fuzzy_Pred_Name", "Matches_With_Base_Name", var_df)
acc_tfidf  = top1_acc_name("TFIDF_Pred_Name", "Matches_With_Base_Name", var_df)

print("\n=== Raw Top-1 Accuracies (no threshold) ===")
print(f"Fuzzy (names)   : {acc_fuzzy:.3f}")
print(f"TF-IDF + cosine : {acc_tfidf:.3f}")


# =========================
# Threshold sweep & recommendation
# =========================
def thresholded_acc(df, score_col, pred_col, thr, scale="0-100"):
    mask = (df[score_col] >= thr) if scale == "0-100" else (df[score_col] >= thr)
    if mask.sum() == 0:
        return 0.0, 0.0
    acc = top1_acc_name(pred_col, "Matches_With_Base_Name", df.loc[mask])
    cov = float(mask.mean())
    return acc, cov

# sweep fuzzy [60..100], cosine [0.2..0.95]
fuzzy_grid = list(range(60, 101, 5))
cos_grid = [x/100 for x in range(20, 96, 5)]

fuzzy_sweep = [(t, *thresholded_acc(var_df, "Fuzzy_Score", "Fuzzy_Pred_Name", t, "0-100")) for t in fuzzy_grid]
cos_sweep   = [(t, *thresholded_acc(var_df, "Cosine_Score", "TFIDF_Pred_Name", t, "0-1")) for t in cos_grid]

best_fuzzy = max(fuzzy_sweep, key=lambda x: x[1]*x[2])  # maximize acc*coverage
best_cos   = max(cos_sweep,   key=lambda x: x[1]*x[2])

print("\n=== Suggested Thresholds (balance = acc × coverage) ===")
print(f"Fuzzy  -> threshold={best_fuzzy[0]}  acc_on_matched={best_fuzzy[1]:.3f}  coverage={best_fuzzy[2]:.3f}")
print(f"Cosine -> threshold={best_cos[0]:.2f} acc_on_matched={best_cos[1]:.3f}  coverage={best_cos[2]:.3f}")

print("\n(For production, good defaults are FUZZY ≥ 85 and COSINE ≥ 0.50.)")


# =========================
# Ensemble decision (per-row)
# =========================
def ensemble_choice(row):
    cos_pass = row["Cosine_Score"] >= COSINE_THRESHOLD
    fz_pass  = row["Fuzzy_Score"]  >= FUZZY_THRESHOLD

    if cos_pass or fz_pass:
        # normalize scores to 0..1 and pick stronger
        norm_cos = row["Cosine_Score"]
        norm_fz  = row["Fuzzy_Score"] / 100.0
        if cos_pass and fz_pass:
            if norm_cos >= norm_fz:
                return row["TFIDF_Pred_ID"], row["TFIDF_Pred_Name"], "cosine"
            else:
                return row["Fuzzy_Pred_ID"], row["Fuzzy_Pred_Name"], "fuzzy"
        elif cos_pass:
            return row["TFIDF_Pred_ID"], row["TFIDF_Pred_Name"], "cosine"
        else:
            return row["Fuzzy_Pred_ID"], row["Fuzzy_Pred_Name"], "fuzzy"
    return None, None, "no_match"

var_df[["Ensemble_Pred_ID", "Ensemble_Pred_Name", "Ensemble_Method"]] = var_df.apply(
    lambda r: pd.Series(ensemble_choice(r)), axis=1
)

# =========================
# Print results
# =========================
cols_show = [
    "Variation",
    "Matches_With_Base_Name",
    "Fuzzy_Pred_Name", "Fuzzy_Score",
    "TFIDF_Pred_Name", "Cosine_Score",
    "Ensemble_Pred_Name", "Ensemble_Method"
]

print("\n=== Full Results ===")
print(var_df[cols_show].to_string(index=False))

mismatch = var_df[
    var_df["Ensemble_Pred_Name"].fillna("").map(normalize_name)
    != var_df["Matches_With_Base_Name"].fillna("").map(normalize_name)
][cols_show]

if len(mismatch):
    print("\n=== Mismatches (post-ensemble, first 15) ===")
    print(mismatch.head(15).to_string(index=False))
else:
    print("\nNo mismatches under current thresholds.")



=== Raw Top-1 Accuracies (no threshold) ===
Fuzzy (names)   : 1.000
TF-IDF + cosine : 0.950

=== Suggested Thresholds (balance = acc × coverage) ===
Fuzzy  -> threshold=60  acc_on_matched=1.000  coverage=0.960
Cosine -> threshold=0.20 acc_on_matched=1.000  coverage=0.950

(For production, good defaults are FUZZY ≥ 85 and COSINE ≥ 0.50.)

=== Full Results ===
         Variation Matches_With_Base_Name   Fuzzy_Pred_Name  Fuzzy_Score   TFIDF_Pred_Name  Cosine_Score Ensemble_Pred_Name Ensemble_Method
      Thomas  King            Thomas King       Thomas King   100.000000       Thomas King      1.000000        Thomas King          cosine
        ThomasKing            Thomas King       Thomas King    57.142857        John Smith      0.000000               None        no_match
      Maria Garcia           Maria Garcia      Maria Garcia   100.000000      Maria Garcia      1.000000       Maria Garcia          cosine
         MaryLewis             Mary Lewis        Mary Lewis    52.631579      